In [0]:
import tensorflow as tf
import numpy as np

In [2]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

import numpy as np
train_data, test_data = imdb['train'], imdb['test']
sentences=[]

for s, l in train_data:
  sentences.append(str(s.numpy()))
  
for s, l in test_data:
  sentences.append(str(s.numpy()))
  

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteNGSTOB/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteNGSTOB/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteNGSTOB/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
vocab_size = 10000
embedding_dim = 100
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(sentences)


In [4]:
lists1 = []
for sentences in sequences:
  l=[]
  for word in sentences:
    l.append(word)
  lists1.append(l)

context_words=[]
target_words =[]

for w in lists1:

  for i in range(3,len(w)-3):
    target_words.append(w[i])
    neighbours = []
    for j in range(i-3,i+4):
      neighbours.append(w[j])
    neighbours.remove(w[i])

    context_words.append(neighbours)

print(target_words[0])
print(context_words[0])


35
[59, 12, 14, 431, 391, 16]


In [0]:
target_words=np.array(target_words)
context_words=np.array(context_words)

In [0]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])


In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=6),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 6, 100)            1000000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 606       
_________________________________________________________________
dense_1 (Dense)              (None, 10000)             70000     
Total params: 1,070,606
Trainable params: 1,070,606
Non-trainable params: 0
_________________________________________________________________


In [8]:
num_epochs = 5
model.fit(context_words,target_words,batch_size = 10000, epochs=num_epochs)

Epoch 1/5
1167/1167 [==============================] - 134s 115ms/step - loss: 6.6144 - accuracy: 0.0519
Epoch 2/5
1167/1167 [==============================] - 133s 114ms/step - loss: 6.3258 - accuracy: 0.0715
Epoch 3/5
1167/1167 [==============================] - 133s 114ms/step - loss: 6.1012 - accuracy: 0.1033
Epoch 4/5
1167/1167 [==============================] - 133s 114ms/step - loss: 5.9815 - accuracy: 0.1145
Epoch 5/5
1167/1167 [==============================] - 134s 114ms/step - loss: 5.9346 - accuracy: 0.1165


In [9]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

(10000, 100)


In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')